# Task 3

## Original exercise number

Exercise 18-6

## Description

The following are the possible hands in poker, in increasing order of value and decreasing order of probability:

*pair*

two cards with the same rank

*two pair*

two pairs of cards with the same rank

*three of a kind*

three cards with the same rank

*straight*

five cards with ranks in sequence (aces can be high or low, so Ace-2-3-4-5 is a straight and so is 10-Jack-Queen-King-Ace, but Queen-King-Ace-2-3 is not.)

*flush*

five cards with the same suit

*full house*

three cards with one rank, two cards with another

*four of a kind*

four cards with the same rank

*straight flush*

five cards in sequence (as defined above) and with the same suit

The goal of this exercise is to estimate the probability of drawing these various hands.

1. Add methods named `haspair`, `hastwopair`, etc. that return `true` or `false` according to whether or not the hand meets the relevant criteria. Your code should work correctly for “hands” that contain any number of cards (although 5 and 7 are the most common sizes).


## Solution

NO GUARANTEE THAT THE SOLUTION WILL WORK OR WORKS CORRECTLY! USE IT AT
YOUR OWN RISK!

### Imports

In [ ]:
using Random

### Structs

In [ ]:
struct Card
    suit :: Int64
    rank :: Int64
    function Card(suit::Int64, rank::Int64)
        @assert(1 ≤ suit ≤ 4, "suit is not between 1 and 4")
        @assert(1 ≤ rank ≤ 13, "rank is not between 1 and 13")
        new(suit, rank)
    end
end

In [ ]:
abstract type CardSet end

In [ ]:
struct Deck <: CardSet
    cards :: Array{Card, 1}
end

function Deck()
    deck = Deck(Card[])
    for suit in 1:4
        for rank in 1:13
            push!(deck.cards, Card(suit, rank))
        end
    end
    deck
end

In [ ]:
struct Hand <: CardSet
    cards :: Array{Card, 1}
    label :: String
end

function Hand(label::String="")
    Hand(Card[], label)
end

### Global Variables

In [ ]:
const suit_names = ["♣", "♦", "♥", "♠"]
const rank_names = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]

### Functions

In [ ]:
function Base.show(io::IO, card::Card)
    print(io, rank_names[card.rank], suit_names[card.suit])
end

In [ ]:
function Base.show(io::IO, cs::CardSet)
    for card in cs.cards
        print(io, card, " ")
    end
end

In [ ]:
function Base.:<(c1::Card, c2::Card)
    (c1.suit, c1.rank) < (c2.suit, c2.rank)
end

In [ ]:
function Base.pop!(cs::CardSet)
    pop!(cs.cards)
end

In [ ]:
function Base.push!(cs::CardSet, card::Card)
    push!(cs.cards, card)
    nothing
end

In [ ]:
function Random.shuffle!(deck::Deck)
    shuffle!(deck.cards)
    deck
end

In [ ]:
function Base.sort!(hand::Hand)
    sort!(hand.cards)
end

In [ ]:
function move!(cs1::CardSet, cs2::CardSet, n::Int)
    @assert 1 ≤ n ≤ length(cs1.cards)
    for i in 1:n
        card = pop!(cs1)
        push!(cs2, card)
    end
    nothing
end

In [ ]:
function deal!(deck::Deck, noOfHands::Int, cardsPerHand::Int)::Vector{Hand}
    @assert (noOfHands >= 1) "number of hands needs to be >= 1"
    @assert (cardsPerHand >= 1) "number of cards per hand needs to be >= 1"
    @assert (noOfHands * cardsPerHand <= length(deck.cards)) "not enough cards in deck"
    hands::Vector{Hand} = []
    tmp::Union{Hand, Nothing} = nothing
    for _ in 1:noOfHands
        tmp = Hand()
        move!(deck, tmp, cardsPerHand)
        push!(hands, tmp)
    end
    return hands
end

In [ ]:
function getCounts(v::Vector{T})::Dict{T,Int} where {T}
    result::Dict{T,Int} = Dict()
    for elt in v
        result[elt] = get(result, elt, 0) + 1
    end
    return result
end

### Functions (solution to task 3.1)

In [ ]:
function getCounts(v::Vector{T})::Dict{T,Int} where {T}
    result::Dict{T,Int} = Dict()
    for elt in v
        result[elt] = get(result, elt, 0) + 1
    end
    return result
end

function getRanks(cs::CardSet)::Vector{Int}
    return map(c -> c.rank, cs.cards)
end

function getSuits(cs::CardSet)::Vector{Int}
    return map(c -> c.rank, cs.cards)
end

function hasNofAkind(hand::Hand, n::Int, ranks::Bool = true)::Bool
    @assert 1 < n < 6 "n must be between 2 and 5"
    theCounts::Dict{Int, Int} = getCounts(ranks ? getRanks(hand) : getSuits(hand))
    return any(v -> v == n, values(theCounts))
end

function hasPair(hand::Hand)::Bool
    return hasNofAkind(hand, 2)
end

function hasTwoPairs(hand::Hand)::Bool
    rankCounts::Dict{Int, Int} = getCounts(getRanks(hand))
    twoPairs::Dict{Int, Int} = Dict(k => v for(k, v) in rankCounts if v == 2)
    return length(twoPairs) == 2
end

function hasThreeOfAKind(hand::Hand)::Bool
    return hasNofAkind(hand, 3)
end

function isV1SubseqOfV2(v1::Vector{Int}, v2::Vector{Int})::Bool
    startInd::Union{Int, Nothing} = findfirst(x -> x == v1[1], v2)
    if isnothing(startInd)
        return false
    else
        return all(map((x, y) -> x == y, v1, v2[startInd:end]))
    end
end

# works for hand with 5 elements (TODO, extend it to more elements)
function hasStraight(hand::Hand)::Bool
    return isV1SubseqOfV2(sort(getRanks(hand)), vcat(1:13, [1]))
end

function hasFlush(hand::Hand)::Bool
    return hasNofAkind(hand, 5, false)
end

function hasFullHouse(hand::Hand)::Bool
    return hasPair(hand) && hasThreeOfAKind(hand)
end

function hasFourOfAKind(hand::Hand)::Bool
    return hasNofAkind(hand, 4)
end

function hasStraightFlush(hand::Hand)::Bool
    return hasFlush(hand) && hasStraight(hand)
end


## Testing